# Classifiez automatiquement des biens de consommation : 
*Notebook mission 2 - Réalisez une classification supervisée d'images*

**Classification superviée d’images via CNN Transfer Learning**

Pourrais-tu réaliser une classification supervisée à partir des images ? Je souhaiterais que tu mettes en place une data augmentation afin d’optimiser le modèle?

In [ ]:
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import scipy
from scipy import stats
import scipy.stats as st

import statsmodels
import statsmodels.api as sm

import sklearn

# pour l'ACP
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix

from sklearn.cluster import KMeans

from sklearn.manifold import TSNE

import timeit
import time

from collections import defaultdict
from collections import Counter

import cv2

import os
from os import listdir

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("scipy version", scipy.__version__)
print("statsmodels version", statsmodels.__version__)

print("sklearn version", sklearn.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

## 1 - Création du modèle de classification

- Charge le modèle pré-entraîné VGG16
  
  + include_top=False : Supprime les couches de classification de VGG16.
  + weights="imagenet" : Charge les poids pré-entraînés sur ImageNet.
  + input_shape=(224, 224, 3) : Définit l’entrée du réseau pour des images couleur de 224×224 pixels.

- Gèle les couches du modèle pré-entraîné

  + Empêche la modification des poids de VGG16 pendant l'entraînement.
  + Seules les couches ajoutées seront entraînées.

- Ajoute des couches supplémentaires

  + GlobalAveragePooling2D : Réduit la dimension de sortie sans trop de paramètres.
  + Dense(256, activation='relu') : Ajoute une couche entièrement connectée.
  + Dropout(0.5) : Réduit le risque d'overfitting en désactivant 50% des neurones.
  + Dense(4, activation='softmax') : La dernière couche de classification avec 7 classes.

- Crée et compile le modèle

  + Définit le modèle final avec comme entrée VGG16 et comme sortie la nouvelle couche de classification.
  + Compile le modèle

In [ ]:
def create_model_fct() :
    # Récupération modèle pré-entraîné
    model0 = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

    # Layer non entraînables = on garde les poids du modèle pré-entraîné
    for layer in model0.layers:
        layer.trainable = False

    # Récupérer la sortie de ce réseau
    x = model0.output
    # Compléter le modèle
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)

    # Définir le nouveau modèle
    model = Model(inputs=model0.input, outputs=predictions)
    # compilation du modèle
    # loss="categorical_crossentropy" → Adapté pour un problème de classification multi-classes.
    # optimizer='rmsprop' → Optimiseur recommandé pour le fine-tuning.
    # metrics=["accuracy"] → Suit la précision du modèle.
    model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=["accuracy"])

    print(model.summary())
    
    return model

## 2 - Approche nouvelle par Dataset avec data augmentation intégrée au modèle

L’approche nouvelle par dataset avec data augmentation intégrée au modèle désigne une méthode dans laquelle l'augmentation des données (data augmentation) est effectuée directement dans le pipeline du modèle, plutôt que d’être réalisée en amont (par prétraitement des images).

- Data Augmentation = technique permettant d’augmenter artificiellement la taille et la diversité d’un dataset en appliquant des transformations (rotation, zoom, bruit, etc.) sur les images d’entraînement. Cela améliore la généralisation du modèle et réduit le surapprentissage (overfitting).

- Intégrée au Modèle = l’augmentation des données n’est pas faite en prétraitement (offline) mais au moment de l'entraînement, via des couches spécifiques ou un pipeline de traitement dynamique.

Avantages :
- Gain de temps : Pas besoin de stocker un grand volume de données augmentées.
- Flexibilité : Les transformations sont appliquées différemment à chaque époque d'entraînement.
- Optimisation mémoire : Les images sont augmentées à la volée, ce qui évite de stocker une grande base de données augmentée.